In [1]:
print("\u2200x ((Lipstick(x) \u2227 In(x, rougeDiorSet) \u2227 In(x, lunarNewYearLimitedEdition) \u2227 VelvetFinish(x)) \u2192 Refillable(x))")
print("\u2200x ((Lipstick(x) \u2227 In(x, rougeDiorSet) \u2227 In(x, lunarNewYearLimitedEdition)) \u2192 (VelvetFinish(x) \u2295 SatinFinish(x))")
print("\u2200x ((Lipstick(x) \u2227 In(x, rougeDiorSet) \u2227 In(x, lunarNewYearLimitedEdition) \u2227 SatinFinish(x)) \u2192 \u00acRosewoodInDescription(x))")
print("\u2200x ((Lipstick(x) \u2227 In(x, rougeDiorSet) \u2227 In(x, lunarNewYearLimitedEdition)) \u2192 (RosewoodInDescription(x) \u2295 \u00acRosewoodInDescription(x)))")
print("Lipstick(rougeDiorColoredLipBalm999) \u2227 In(rougeDiorColoredLipBalm999, rougeDiorSet) \u2227 In(rougeDiorColoredLipBalm999, lunarNewYearLimitedEdition) \u2227 (RosewoodInDescription(rougeDiorColoredLipBalm999) \u2295 VelvetFinish(rougeDiorColoredLipBalm999))")

∀x ((Lipstick(x) ∧ In(x, rougeDiorSet) ∧ In(x, lunarNewYearLimitedEdition) ∧ VelvetFinish(x)) → Refillable(x))
∀x ((Lipstick(x) ∧ In(x, rougeDiorSet) ∧ In(x, lunarNewYearLimitedEdition)) → (VelvetFinish(x) ⊕ SatinFinish(x))
∀x ((Lipstick(x) ∧ In(x, rougeDiorSet) ∧ In(x, lunarNewYearLimitedEdition) ∧ SatinFinish(x)) → ¬RosewoodInDescription(x))
∀x ((Lipstick(x) ∧ In(x, rougeDiorSet) ∧ In(x, lunarNewYearLimitedEdition)) → (RosewoodInDescription(x) ⊕ ¬RosewoodInDescription(x)))
Lipstick(rougeDiorColoredLipBalm999) ∧ In(rougeDiorColoredLipBalm999, rougeDiorSet) ∧ In(rougeDiorColoredLipBalm999, lunarNewYearLimitedEdition) ∧ (RosewoodInDescription(rougeDiorColoredLipBalm999) ⊕ VelvetFinish(rougeDiorColoredLipBalm999))


## Predicates

In [56]:
import re
import json

reg = re.compile(r'(?!x|y|z|w)\b\w+\s*\([^\)]+\)')

with open('data/FOLIO/train.json', 'r') as f:
    train = json.load(f)

In [57]:
args_map = {
    1: 'x',
    2: 'x, y',
    3: 'x, y, z',
    4: 'x, y, z, w'
}

In [58]:
def clean_predicate(predicate):
    name = predicate.split('(')[0].strip()
    arguments = [p.split() for p in predicate.split('(')[1][:-1].split(',')]
    
    n_arguments = len(arguments)
    
    clean_predicate = name + '(' + args_map[n_arguments] + ')'

    return clean_predicate

In [59]:
for point in train:
    formulas = ' '.join(point['context_fol'])
    preds = list(set(re.findall(reg, formulas)))
    
    clean_preds = list(set(clean_predicate(pred) for pred in preds))
    
    point['logic_predicates'] = clean_preds

In [60]:
with open('data/FOLIO/train.json', 'w') as f:
    json.dump(train, f, indent=2, ensure_ascii=False)

## The rest

In [3]:
import json

In [7]:
with open('outputs/logic_predicates/FOLIO_dev_gpt-3.5-turbo.json', 'r') as f:
    preds = json.load(f)

In [8]:
with open('outputs/logic_programs/FOLIO_dev_gpt-3.5-turbo_static.json', 'r') as f:
    progs = json.load(f)

In [13]:
preds.keys()

dict_keys(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '99', '100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '110', '111', '113', '115', '116', '117', '118', '119', '120', '121', '122', '123', '124', '126', '127', '128', '129', '130', '131', '132', '133', '134', '135', '136', '137', '140', '141', '142', '143', '144', '145', '146', '147', '148', '149', '150', '151', '152', '153', '154', '155', '156', '157', '158', '159', '160', '161', '162', '163', '164', '165', '1

In [17]:
for prog in progs:
    if str(prog['id']) in preds:
        prog['predicates'] = preds[str(prog['id'])]['logic_predicates']
    else:
        prog['predicates'] = []

In [18]:
progs

[{'id': 0,
  'context': ['If people perform in school talent shows often, then they attend and are very engaged with school events.',
   'People either perform in school talent shows often or are inactive and disinterested members of their community.',
   'If people chaperone high school dances, then they are not students who attend the school.',
   'All people who are inactive and disinterested members of their community chaperone high school dances.',
   'All young children and teenagers who wish to further their academic careers and educational opportunities are students who attend the school.',
   'Bonnie either both attends and is very engaged with school events and is a student who attends the school, or she neither attends and is very engaged with school events nor is a student who attends the school. '],
  'question': 'Bonnie performs in school talent shows often.',
  'answer': 'C',
  'raw_logic_programs': ['First-Order-Logic Premises:\n∀x (Perform(x) → (Attend(x) ∧ Engaged(x))

## GCD

In [11]:
with open('models/grammars/FOLIO.gbnf', 'r') as f:
    grammar = f.read()

In [12]:
from llama_cpp import Llama
from llama_cpp.llama import LlamaGrammar
# callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])
n_gpu_layers = -1  # The number of layers to put on the GPU. The rest will be on the CPU. If you don't know how many layers there are, you can use -1 to move all to GPU.
n_batch = 1024  # Should be between 1 and n_ctx, consider the amount of RAM of your Apple Silicon Chip.
# Make sure the model path is correct for your system!
llm = Llama(
    # model_path="models/mistral-7b-instruct-v0.2-code-ft.Q5_K_M.gguf",
    model_path="GCD/llms/nous-hermes-2-solar-10.7b.Q6_K.gguf",
    n_gpu_layers=n_gpu_layers,
    n_batch=n_batch,
    f16_kv=True,  # MUST set to True, otherwise you will run into problem after a couple of calls
    # callback_manager=callback_manager,
    # verbose=True,  # Verbose is required to pass to the callback manager
    verbose=False,
)


In [13]:

user = "Say Hello World!"

In [20]:


result = llm.create_chat_completion(
messages=[
    {"role": "user", "content": user}
    ],
# max_tokens=max_tokens,  # Generate up to 512 token

)

KeyboardInterrupt: 

In [19]:
result['choices'][0]['message']['content']

'\n\n### Description\n\nThis is a simple example of how to use the `INST` tag. It will output "Hello World!" when rendered.'

## Logic

In [4]:
import re
import json
import os
from copy import deepcopy
from nltk.inference.prover9 import Prover9Command
from nltk.sem.logic import *

from models.symbolic_solvers.fol_solver.Formula_util import FOL_Formula
from models.symbolic_solvers.fol_solver.fol_prover9_parser import Prover9_FOL_Formula

from sentence_transformers import SentenceTransformer, util
import numpy as np
from tqdm.autonotebook import tqdm

os.environ['PROVER9'] = './models/symbolic_solvers/Prover9/bin'

Samples removed from dev:

- 47
- 66
- 67
- 68
- 98
- 112
- 114
- 125
- 138
- 139
- 171

Samples removed from train:

- 691

In [11]:
original = Expression.fromstring('all x.(Roundel(x) -> -Higher(x) -> Lower(x))')
new = Expression.fromstring('all x y.exists z(Know(x, z) & Know(y, z) & UniversalLanguage(z)) -> Communicate(x, y)')

original.equiv(new)

False

(all x.(GSCjasdhjvfd))

In [14]:
context_fol= [
      "∀x (RussianBank(x) → ¬ApplePay(x))",
      "∀x (Mir(x) → RussianBank(x))",
      "∃x (International(x) → ApplePay(x))",
      "∀x (SocialRussia(x) → Mir(x))",
    ]

question_fol = "ApplePay(boapaymentcards)"

In [15]:
ass= []

for s in context_fol:
    fol_formula = FOL_Formula(s)

    if fol_formula.is_valid:
        prover9_formula = Prover9_FOL_Formula(fol_formula).formula
        # 
        ass.append(Expression.fromstring(prover9_formula))
    else:
        print(f'error: ', s)

In [16]:
goal = Expression.fromstring(Prover9_FOL_Formula(FOL_Formula(question_fol)).formula)

In [17]:
prover = Prover9Command(goal, ass)

In [71]:
goal = NegatedExpression(goal)

In [80]:
prover = Prover9Command(goal, ass)

In [18]:
prover.prove()

False

In [36]:
prover.proof(simplify=False)

'============================== Prover9 ===============================\nProver9 (64) version 2009-11A, November 2009.\nProcess 60900 was started by feder on MacBook-Pro-2.local,\nSun May  5 17:13:18 2024\nThe command was "./models/symbolic_solvers/Prover9/bin/prover9".\n============================== end of head ===========================\n\n============================== INPUT =================================\nassign(max_seconds,60).\nclear(auto_denials).\n\nformulas(assumptions).\n(all x (Meeting(x) -> AppearInCompany(x))).\n(all x (LunchInCompany(x) -> Meetings(x))).\n(all x (LunchInCompany(x) & -LunchAtHome(x) | -LunchInCompany(x) & LunchAtHome(x))).\n(all x (LunchAtHome(x) -> WorkRemotelyFromHome(x))).\n(all x (InOtherCountries(x) -> WorkRemotelyFromHome(x))).\n(all x (Manager(x) -> -WorkRemotelyFromHome(x))).\nManager(James) & --AppearInCompany(James) | -Manager(James) & -AppearInCompany(James).\nend_of_list.\n\nformulas(goals).\nManager(James) & -InOtherCountries(James) | -Ma

In [5]:
ass

[<AllExpression all x.((GrandSlamChampion(x) & -OscarNominatedActor(x)) | (-GrandSlamChampion(x) & OscarNominatedActor(x)))>]

In [ ]:
with open('old/outputs/logic_inference/FOLIO_dev_gpt-3.5-turbo_static_text_backup-LLM.json', 'r') as f:
    old_inference = json.load(f)

In [ ]:
with open('outputs/logic_inference/FOLIO_dev_gpt-3.5-turbo_static_text_backup-LLM.json', 'r') as f:
    new_inference = json.load(f)

## Conversion Datasets

In [ ]:
with open('parsing/new_formulas_v0/FOLIO_dev_gpt-3.5-turbo_static_text.json', 'r') as j:
    folio_v0_train_parsed = json.load(j)

In [ ]:
with open('data/FOLIO/dev.json', 'r') as j:
    folio_v0_train = json.load(j)

In [ ]:
folio_v0_train_dict = {val['id']: val for val in folio_v0_train}

In [ ]:
folio_v0_train_parsed_dict = {val['id']: val for val in folio_v0_train_parsed}

In [ ]:
for id in folio_v0_train_dict.keys():
    folio_v0_train_dict[id]['context_fol'] = folio_v0_train_parsed_dict[id]['assumptions']
    folio_v0_train_dict[id]['question_fol'] = folio_v0_train_parsed_dict[id]['goal']

In [ ]:
new_folio_v0_train = [val for key, val in folio_v0_train_dict.items()]

In [ ]:
with open('parsed_data/FOLIO/dev.json', 'w') as j:
    json.dump(new_folio_v0_train, j)